## Q1

Three different types of orders inherit the class of Order with parameters id, symbol, quantity, side and timestamp. The key differences among them are that Limit and IOC order have one extra element which is a non-negative price while Market order does not have since it fills regardless to the price.

Aware of these conditions, we can explain how the matching engine works. The matching engine works to match and execute the orders with Limit orders as the base units in the order book. When the bid/ask order books are empty, we will insert the available Limit order to wait for the next matching order. Then, we sort the bid book ascending by price and timestamp, and we sort the ask book descending by price and ascending by timestamp. 

For the Buy Limit order, we want the highest price that is smaller than the order price in the ask book and try to fill the order quantity. If we cannot fill the order with this ask, we will fill the rest quantity by finding the next available price under the same algorithm. The filled portion of both the order and the ask in the book will be added to the Filled Orders. For Sell Limit order, we follow the same process but this time we find in the bid book and want the lowest price that is larger than the order price in the bid book. For any portion of the unfilled order, we will hold them in the corresponding book. That is to put Buy orders in the bid book and Sell orders in the ask book. 

For the Buy Market order, we do not care about the prices try to fill the order quantity from the highest price the ask book. If we cannot fill the order with this ask, we will fill the rest quantity by finding the next available price under the same algorithm. The filled portion of both the order and the ask in the book will be added to the Filled Orders. For Sell Market order, we follow the same process but this time we find in the bid book and fill from the lowest price in the bid book. We don't hold Market orders as there is no available price for it. Also, as long as the quantity is sufficient in the order book, Market can always be filled. 

For the IOC orders, it works the same as how we fill the Limit orders except for one difference. That is we have to fill part or all of the IOC orders immediately and cannot hold the rest portion in the order book to wait. 

## Q2

In [79]:
import time

from enum import Enum
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2


class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass


from abc import ABC


class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC
    

class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit = False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit
        



class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []
        # These are the order books you are given and expected to use for matching the orders below

    # Note: As you implement the following functions keep in mind that these enums are available:
#     class OrderType(Enum):
#         LIMIT = 1
#         MARKET = 2
#         IOC = 3

#     class OrderSide(Enum):
#         BUY = 1
#         SELL = 2

    def handle_order(self, order):
        # Implement this function
        # In this function you need to call different functions from the matching engine
        # depending on the type of order you are given
        if order.type == 1: 
            return self.handle_limit_order(order) 
        elif order.type == 2: 
            return self.handle_market_order(order) 
        elif order.type == 3: 
            return self.handle_ioc_order(order) 
        else:
        # You need to raise the following error if the type of order is ambiguous
            raise UndefinedOrderType("Undefined Order Type!") 

    
    def handle_limit_order(self, order): 
        # Implement this function
        # Keep in mind what happens to the orders in the limit order books when orders get filled
        # or if there are no crosses from this order
        # in other words, handle_limit_order accepts an arbitrary limit order that can either be 
        # filled if the limit order price crosses the book, or placed in the book. If the latter, 
        # pass the order to insert_limit_order below. 
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        
        # The filled orders are expected to be the return variable (list)
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                self.insert_limit_order(order) 
            else: 
                while True: 
                    if order.price < self.ask_book[0].price: 
                        self.insert_limit_order(order) 
                        break
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.ask_book)): 
                        if order.price < self.ask_book[i].price: 
                            match = i-1
                            break
                    x = self.ask_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, order.price, order.side, order.time, True)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, order.price, order.side, order.time, True)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.ask_book.pop(match)
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                self.insert_limit_order(order) 
            else: 
                while True: 
                    if order.price > self.bid_book[0].price: 
                        self.insert_limit_order(order) 
                        break
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.bid_book)): 
                        if order.price > self.bid_book[i].price: 
                            match = i-1
                            break
                    x = self.bid_book[match]
                    if self.bid_book[match].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, order.price, order.side, order.time, True)) 
                        self.bid_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, order.price, order.side, order.time, True)) 
                        order.quantity -= self.bid_book[match].quantity 
                        self.bid_book.pop(match)     
        # You need to raise the following error if the side the order is for is ambiguous
        else:   
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders 


    def handle_market_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        # The filled orders are expected to be the return variable (list)
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                return filled_orders
            else: 
                while True: 
                    if order.quantity == 0 or len(self.ask_book) == 0: 
                        break 
                    x = self.ask_book[0]
                    if self.ask_book[0].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.ask_book[0].quantity -= order.quantity 
                        order.quantity = 0 
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.ask_book[0].quantity 
                        self.ask_book.pop(0)  
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                return filled_orders
            else: 
                while True: 
                    if order.quantity == 0 or len(self.bid_book) == 0: 
                        break 
                    x = self.bid_book[0]
                    if self.bid_book[0].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.bid_book[0].quantity -= order.quantity 
                        order.quantity = 0 
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.bid_book[0].quantity 
                        self.bid_book.pop(0)                 
        # You need to raise the following error if the side the order is for is ambiguous
        else: 
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders
        

    def handle_ioc_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the ioc order need to be inserted into the above list
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                return 
            else: 
                while True: 
                    if order.price < self.ask_book[0].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.ask_book)): 
                        if order.price < self.ask_book[i].price: 
                            match = i-1
                    x = self.ask_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.cancel_order(self.ask_book[match].id) 
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                return 
            else: 
                while True: 
                    if order.price > self.bid_book[0].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.bid_book)): 
                        if order.price > self.bid_book[i].price: 
                            match = i-1
                    x = self.bid_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.cancel_order(self.ask_book[match].id)      
        # You need to raise the following error if the side the order is for is ambiguous
        else:   
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders 
        # The filled orders are expected to be the return variable (list)


    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        # Implement this function 
        # this function's sole puporse is to place limit orders in the book that are guaranteed
        # to not immediately fill
        if order.side == OrderSide.BUY:
            self.bid_book += [order]
            self.bid_book.sort(key=lambda x: (-1*x.price,x.time))
        elif order.side == OrderSide.SELL:
            self.ask_book += [order]
            self.ask_book.sort(key=lambda x: (x.price,x.time))
        else: 
            raise UndefinedOrderSide("Undefined Order Side!") 
        # You need to raise the following error if the side the order is for is ambiguous

    def amend_quantity(self, id, quantity):
        # Implement this function
        # Hint: Remember that there are two order books, one on the bid side and one on the ask side
        
        # You need to raise the following error if the user attempts to modify an order
        # with a quantity that's greater than given in the existing order 
        for i in range(len(self.bid_book)): 
            if self.bid_book[i].id == id: 
                if self.bid_book[i].quantity >= quantity: 
                    self.bid_book[i].quantity = quantity
                else: 
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!") 
        for i in range(len(self.ask_book)): 
            if self.ask_book[i].id == id: 
                if self.ask_book[i].quantity >= quantity: 
                    self.ask_book[i].quantity = quantity
                else: 
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
        return False 
    
    def cancel_order(self, id):
        # Implement this function
        # Think about the changes you need to make in the order book based on the parameters given
        for i in range(len(self.ask_book)): 
            if self.ask_book[i].id == id: 
                self.ask_book.pop(i) 
                break 
        
        for i in range(len(self.bid_book)): 
            if self.bid_book[i].id == id: 
                self.bid_book.pop(i) 
                break
        return False
import unittest

class TestOrderBook(unittest.TestCase):

    def test_insert_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        self.assertEqual(matching_engine.bid_book[0].quantity, 10)
        self.assertEqual(matching_engine.bid_book[0].price, 10)
    
    def test_handle_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(3, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        self.assertEqual(matching_engine.bid_book[0].price, 15)
        self.assertEqual(matching_engine.bid_book[1].quantity, 10)

        order_sell = LimitOrder(4, "S", 14, 8, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_limit_order(order_sell)

        self.assertEqual(matching_engine.bid_book[0].quantity, 6)
        self.assertEqual(filled_orders[0].id, 3)
        self.assertEqual(filled_orders[0].price, 15)
        self.assertEqual(filled_orders[2].id, 1)
        self.assertEqual(filled_orders[2].price, 10)
    
    def test_handle_market_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 6, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = MarketOrder(5, "S", 5, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_market_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(filled_orders[0].price, 10) 

    def test_handle_ioc_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_ioc_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(len(filled_orders), 0)
    
    def test_amend_quantity(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.amend_quantity(2, 8)
        self.assertEqual(matching_engine.bid_book[0].quantity, 8)
    
    def test_cancel_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.cancel_order(1)
        self.assertEqual(matching_engine.bid_book[0].id, 2)

import io
import __main__
suite = unittest.TestLoader().loadTestsFromModule(__main__)
buf = io.StringIO()
unittest.TextTestRunner(stream=buf, verbosity=2).run(suite)
buf = buf.getvalue().split("\n")
for test in buf:
	if test.startswith("test"):
		print(test)

test_amend_quantity (__main__.TestOrderBook) ... ok
test_cancel_order (__main__.TestOrderBook) ... ok
test_handle_ioc_order (__main__.TestOrderBook) ... ok
test_handle_limit_order (__main__.TestOrderBook) ... ok
test_handle_market_order (__main__.TestOrderBook) ... ok
test_insert_limit_order (__main__.TestOrderBook) ... ok


## Q3

In [ ]:
import time

from enum import Enum
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2


class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass


from abc import ABC


class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC
    

class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit = False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit
        


# Paste in your implementation for the matching engine below

# ----------------------------------------------------------
# PASTE MATCHING ENGINE FROM Q2 HERE
class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []
        # These are the order books you are given and expected to use for matching the orders below

    # Note: As you implement the following functions keep in mind that these enums are available:
#     class OrderType(Enum):
#         LIMIT = 1
#         MARKET = 2
#         IOC = 3

#     class OrderSide(Enum):
#         BUY = 1
#         SELL = 2

    def handle_order(self, order):
        # Implement this function
        # In this function you need to call different functions from the matching engine
        # depending on the type of order you are given
        if order.type == 1: 
            return self.handle_limit_order(order) 
        elif order.type == 2: 
            return self.handle_market_order(order) 
        elif order.type == 3: 
            return self.handle_ioc_order(order) 
        else:
        # You need to raise the following error if the type of order is ambiguous
            raise UndefinedOrderType("Undefined Order Type!") 

    
    def handle_limit_order(self, order): 
        # Implement this function
        # Keep in mind what happens to the orders in the limit order books when orders get filled
        # or if there are no crosses from this order
        # in other words, handle_limit_order accepts an arbitrary limit order that can either be 
        # filled if the limit order price crosses the book, or placed in the book. If the latter, 
        # pass the order to insert_limit_order below. 
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        
        # The filled orders are expected to be the return variable (list)
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                self.insert_limit_order(order) 
            else: 
                while True: 
                    if order.price < self.ask_book[0].price: 
                        self.insert_limit_order(order) 
                        break
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.ask_book)): 
                        if order.price < self.ask_book[i].price: 
                            match = i-1
                            break
                    x = self.ask_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, order.price, order.side, order.time, True)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, order.price, order.side, order.time, True)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.ask_book.pop(match)
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                self.insert_limit_order(order) 
            else: 
                while True: 
                    if order.price > self.bid_book[0].price: 
                        self.insert_limit_order(order) 
                        break
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.bid_book)): 
                        if order.price > self.bid_book[i].price: 
                            match = i-1
                            break
                    x = self.bid_book[match]
                    if self.bid_book[match].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, order.price, order.side, order.time, True)) 
                        self.bid_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, order.price, order.side, order.time, True)) 
                        order.quantity -= self.bid_book[match].quantity 
                        self.bid_book.pop(match)     
        # You need to raise the following error if the side the order is for is ambiguous
        else:   
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders 


    def handle_market_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        # The filled orders are expected to be the return variable (list)
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                return filled_orders
            else: 
                while True: 
                    if order.quantity == 0 or len(self.ask_book) == 0: 
                        break 
                    x = self.ask_book[0]
                    if self.ask_book[0].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.ask_book[0].quantity -= order.quantity 
                        order.quantity = 0 
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.ask_book[0].quantity 
                        self.ask_book.pop(0)  
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                return filled_orders
            else: 
                while True: 
                    if order.quantity == 0 or len(self.bid_book) == 0: 
                        break 
                    x = self.bid_book[0]
                    if self.bid_book[0].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.bid_book[0].quantity -= order.quantity 
                        order.quantity = 0 
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.bid_book[0].quantity 
                        self.bid_book.pop(0)                 
        # You need to raise the following error if the side the order is for is ambiguous
        else: 
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders
        

    def handle_ioc_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the ioc order need to be inserted into the above list
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                return 
            else: 
                while True: 
                    if order.price < self.ask_book[0].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.ask_book)): 
                        if order.price < self.ask_book[i].price: 
                            match = i-1
                    x = self.ask_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.cancel_order(self.ask_book[match].id) 
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                return 
            else: 
                while True: 
                    if order.price > self.bid_book[0].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.bid_book)): 
                        if order.price > self.bid_book[i].price: 
                            match = i-1
                    x = self.bid_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.cancel_order(self.ask_book[match].id)      
        # You need to raise the following error if the side the order is for is ambiguous
        else:   
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders 
        # The filled orders are expected to be the return variable (list)


    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        # Implement this function 
        # this function's sole puporse is to place limit orders in the book that are guaranteed
        # to not immediately fill
        if order.side == OrderSide.BUY:
            self.bid_book += [order]
            self.bid_book.sort(key=lambda x: (-1*x.price,x.time))
        elif order.side == OrderSide.SELL:
            self.ask_book += [order]
            self.ask_book.sort(key=lambda x: (x.price,x.time))
        else: 
            raise UndefinedOrderSide("Undefined Order Side!") 
        # You need to raise the following error if the side the order is for is ambiguous

    def amend_quantity(self, id, quantity):
        # Implement this function
        # Hint: Remember that there are two order books, one on the bid side and one on the ask side
        
        # You need to raise the following error if the user attempts to modify an order
        # with a quantity that's greater than given in the existing order 
        for i in range(len(self.bid_book)): 
            if self.bid_book[i].id == id: 
                if self.bid_book[i].quantity >= quantity: 
                    self.bid_book[i].quantity = quantity
                else: 
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!") 
        for i in range(len(self.ask_book)): 
            if self.ask_book[i].id == id: 
                if self.ask_book[i].quantity >= quantity: 
                    self.ask_book[i].quantity = quantity
                else: 
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
        return False 
    
    def cancel_order(self, id):
        # Implement this function
        # Think about the changes you need to make in the order book based on the parameters given
        for i in range(len(self.ask_book)): 
            if self.ask_book[i].id == id: 
                self.ask_book.pop(i) 
                break 
        
        for i in range(len(self.bid_book)): 
            if self.bid_book[i].id == id: 
                self.bid_book.pop(i) 
                break
        return False
#-----------------------------------------------------------

# You need to build additional unittests for your implementation
import unittest

class TestOrderBook(unittest.TestCase):
    
    # The unittests must start with "test", change the function name to something that makes sense
    def test_handle_limit_order_extra(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.SELL, time.time())
        matching_engine.insert_limit_order(order)

        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.SELL, time.time())
        order_2 = LimitOrder(3, "S", 15, 15, OrderSide.SELL, time.time())
        order_3 = LimitOrder(4, "S", 7, 10, OrderSide.SELL, time.time())
        order_4 = LimitOrder(5, "S", 5, 12, OrderSide.SELL, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)
        matching_engine.handle_limit_order(order_3)
        matching_engine.handle_limit_order(order_4)

        self.assertEqual(matching_engine.ask_book[0].price, 10)
        self.assertEqual(matching_engine.ask_book[1].quantity, 5)
        self.assertEqual(matching_engine.ask_book[2].quantity, 7)
        self.assertEqual(matching_engine.ask_book[3].id, 6)
        self.assertEqual(matching_engine.ask_book[4].price, 15)
        
        order_buy1 = LimitOrder(6, "S", 14, 12, OrderSide.BUY, time.time())
        filled_orders = matching_engine.handle_limit_order(order_buy1)
        self.assertEqual(matching_engine.ask_book[0].quantity, 6)
        self.assertEqual(filled_orders[0].id, 5)
        self.assertEqual(filled_orders[0].price, 12)
        self.assertEqual(filled_orders[2].id, 1)
        self.assertEqual(filled_orders[2].price, 10)

        order_buy2 = LimitOrder(7, "S", 5, 5, OrderSide.BUY, time.time())
        filled_orders2 = matching_engine.handle_limit_order(order_buy2)
        self.assertEqual(matching_engine.bid_book[0].quantity, 5)
        self.assertEqual(matching_engine.bid_book[0].id, 7)
        
    
    def test_cancel_order_extra(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.SELL, time.time())
        matching_engine.insert_limit_order(order_1)
        order_2 = LimitOrder(3, "S", 15, 15, OrderSide.SELL, time.time())
        matching_engine.handle_limit_order(order_2)
        self.assertEqual(matching_engine.ask_book[0].id , 2)
        matching_engine.cancel_order(2)
        self.assertEqual(matching_engine.ask_book[0].id ,3)
    
    def test_handle_ioc_order_extra(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = IOCOrder(3, "S", 4, 5, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_ioc_order(order)
        self.assertEqual(matching_engine.bid_book[1].quantity, 1)
        self.assertEqual(len(filled_orders), 2)
    
        
    
    def test_handle_market_order_extra(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 14, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 12, OrderSide.BUY, time.time())
        order_3 = LimitOrder(3, "S", 8, 11, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)
        matching_engine.handle_limit_order(order_3)
        order = MarketOrder(4, "S", 15, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_market_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 3)
        self.assertEqual(len(filled_orders), 6)
    
# A few example unittests are provided below

    def test_insert_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        self.assertEqual(matching_engine.bid_book[0].quantity, 10)
        self.assertEqual(matching_engine.bid_book[0].price, 10)
    
    def test_handle_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(3, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        self.assertEqual(matching_engine.bid_book[0].price, 15)
        self.assertEqual(matching_engine.bid_book[1].quantity, 10)

        order_sell = LimitOrder(4, "S", 14, 8, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_limit_order(order_sell)

        self.assertEqual(matching_engine.bid_book[0].quantity, 6)
        self.assertEqual(filled_orders[0].id, 3)
        self.assertEqual(filled_orders[0].price, 15)
        self.assertEqual(filled_orders[2].id, 1)
        self.assertEqual(filled_orders[2].price, 10)
    
    def test_handle_market_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 6, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = MarketOrder(5, "S", 5, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_market_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(filled_orders[0].price, 10)

    def test_handle_ioc_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_ioc_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(len(filled_orders), 0)
    
    def test_amend_quantity(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.amend_quantity(2, 8)
        self.assertEqual(matching_engine.bid_book[0].quantity, 8)
    
    def test_cancel_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.cancel_order(1)
        self.assertEqual(matching_engine.bid_book[0].id, 2)

import io
import __main__
suite = unittest.TestLoader().loadTestsFromModule(__main__)
buf = io.StringIO()
unittest.TextTestRunner(stream=buf, verbosity=2).run(suite)
buf = buf.getvalue().split("\n")
sum=0
for test in buf:
	if test.startswith("test"):
		sum+=1
        
print("You have %d unit tests" % (sum))



## Q4

In [ ]:
from collections import deque
import time
import random
from abc import ABC



from enum import Enum
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2

class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass

class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC


class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit=False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit

        
trader_to_exchange = deque()
exchange_to_trader = [deque() for _ in range(100)]
# Above you are given two deques where the orders submitted to the exchange and back to the trader
# are expected to be populated by the trading exchange simulator
# The first is trader_to_exchange, a deque of orders to be populated for the exchange to execute
# The second is a list of 100 deques exchange_to_trader, which are acknowledgements from the exchange
# to each of the 100 traders for trades executed on their behalf

# Below you have an implementation of a simulated thread to be used where each trader is a separate thread
class MyThread:
    list_of_threads=[]
    def __init__(self,id='NoID'):
        MyThread.list_of_threads.append(self)
        self.is_started=False
        self.id = id
    def start(self):
        self.is_started = True
    def join(self):
        if str(self.id) == 'NoID':
            pass
        else:
            print('Trader ' + str(self.id) + ' will be waited')
        pass

order_id = 0


# Paste in your implementation for the matching engine below

# ----------------------------------------------------------
# PASTE MATCHING ENGINE FROM Q2 HERE
#-----------------------------------------------------------

class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []
        # These are the order books you are given and expected to use for matching the orders below

    # Note: As you implement the following functions keep in mind that these enums are available:
#     class OrderType(Enum):
#         LIMIT = 1
#         MARKET = 2
#         IOC = 3

#     class OrderSide(Enum):
#         BUY = 1
#         SELL = 2

    def handle_order(self, order):
        # Implement this function
        # In this function you need to call different functions from the matching engine
        # depending on the type of order you are given
        if order.type == 1: 
            return self.handle_limit_order(order) 
        elif order.type == 2: 
            return self.handle_market_order(order) 
        elif order.type == 3: 
            return self.handle_ioc_order(order) 
        else:
        # You need to raise the following error if the type of order is ambiguous
            raise UndefinedOrderType("Undefined Order Type!") 

    
    def handle_limit_order(self, order): 
        # Implement this function
        # Keep in mind what happens to the orders in the limit order books when orders get filled
        # or if there are no crosses from this order
        # in other words, handle_limit_order accepts an arbitrary limit order that can either be 
        # filled if the limit order price crosses the book, or placed in the book. If the latter, 
        # pass the order to insert_limit_order below. 
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        
        # The filled orders are expected to be the return variable (list)
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                self.insert_limit_order(order) 
            else: 
                while True: 
                    if order.price < self.ask_book[0].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.ask_book)): 
                        if order.price >= self.ask_book[i].price: 
                            match = i-1 
                    x = self.ask_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.cancel_order(self.ask_book[match].id) 
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                self.insert_limit_order(order) 
            else: 
                while True: 
                    if order.price > self.bid_book[-1].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.bid_book)): 
                        if order.price >= self.bid_book[i].price: 
                            match = i-1 
                    x = self.bid_book[match]
                    if x.quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        x.quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= x.quantity 
                        self.cancel_order(x.id)      
        # You need to raise the following error if the side the order is for is ambiguous
        else:   
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders 


    def handle_market_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        # The filled orders are expected to be the return variable (list)
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                return filled_orders
            else: 
                while True: 
                    if order.quantity == 0: 
                        break 
                    i = 0 
                    x = self.ask_book[i]
                    if x.quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.ask_book[i].quantity -= order.quantity 
                        order.quantity = 0 
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.ask_book[i].quantity 
                        self.ask_book.pop(i)  
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                return filled_orders
            else: 
                while True: 
                    if order.quantity == 0: 
                        break 
                    i = 0 
                    x = self.bid_book[i]
                    if x.quantity >= order.quantity: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time, True)) 
                        self.bid_book[i].quantity -= order.quantity 
                        order.quantity = 0 
                        break
                    else: 
                        filled_orders.append(FilledOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time, True)) 
                        filled_orders.append(FilledOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time, True)) 
                        order.quantity -= self.bid_book[i].quantity 
                        self.bid_book.pop(i)                 
        # You need to raise the following error if the side the order is for is ambiguous
        else: 
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders
        

    def handle_ioc_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the ioc order need to be inserted into the above list
        if order.side == OrderSide.BUY: 
            if len(self.ask_book) == 0: 
                return 
            else: 
                while True: 
                    if order.price < self.ask_book[0].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.ask_book)): 
                        if order.price >= self.ask_book[i].price: 
                            match = i-1 
                    x = self.ask_book[match]
                    if self.ask_book[match].quantity >= order.quantity: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time)) 
                        self.ask_book[match].quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time)) 
                        order.quantity -= self.ask_book[match].quantity 
                        self.cancel_order(self.ask_book[match].id) 
        elif order.side == OrderSide.SELL: 
            if len(self.bid_book) == 0: 
                return 
            else: 
                while True: 
                    if order.price > self.bid_book[-1].price: 
                        break 
                    if order.quantity == 0: 
                        break 
                    match = 0 
                    for i in range(len(self.bid_book)): 
                        if order.price >= self.bid_book[i].price: 
                            match = i-1 
                    x = self.bid_book[match]
                    if x.quantity >= order.quantity: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, order.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, order.quantity, x.price, order.side, order.time)) 
                        x.quantity -= order.quantity 
                        order.quantity = 0
                    else: 
                        filled_orders.append(LimitOrder(x.id, x.symbol, x.quantity, x.price, x.side, x.time)) 
                        filled_orders.append(LimitOrder(order.id, order.symbol, x.quantity, x.price, order.side, order.time)) 
                        order.quantity -= x.quantity 
                        self.cancel_order(x.id)      
        # You need to raise the following error if the side the order is for is ambiguous
        else:   
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders 
        # The filled orders are expected to be the return variable (list)


    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        # Implement this function 
        # this function's sole puporse is to place limit orders in the book that are guaranteed
        # to not immediately fill
        if order.side == OrderSide.BUY:
            self.bid_book += [order]
            self.bid_book.sort(key=lambda x: (-1*x.price,x.time))
        elif order.side == OrderSide.SELL:
            self.ask_book += [order]
            self.ask_book.sort(key=lambda x: (x.price,x.time))
        else: 
            raise UndefinedOrderSide("Undefined Order Side!") 
        # You need to raise the following error if the side the order is for is ambiguous

    def amend_quantity(self, id, quantity):
        # Implement this function
        # Hint: Remember that there are two order books, one on the bid side and one on the ask side
        
        # You need to raise the following error if the user attempts to modify an order
        # with a quantity that's greater than given in the existing order 
        for i in range(len(self.bid_book)): 
            if self.bid_book[i].id == id: 
                if self.bid_book[i].quantity >= quantity: 
                    self.bid_book[i].quantity = quantity
                else: 
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!") 
        for i in range(len(self.ask_book)): 
            if self.ask_book[i].id == id: 
                if self.ask_book[i].quantity >= quantity: 
                    self.ask_book[i].quantity = quantity
                else: 
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
        return False 
    
    def cancel_order(self, id):
        # Implement this function
        # Think about the changes you need to make in the order book based on the parameters given
        for i in range(len(self.ask_book)): 
            if self.ask_book[i].id == id: 
                self.ask_book.pop(i) 
                break 
        
        for i in range(len(self.bid_book)): 
            if self.bid_book[i].id == id: 
                self.bid_book.pop(i) 
                break
        return False


# Each trader can take a separate action chosen from the list below:

# Actions:
# 1 - Place New Order/Order Filled
# 2 - Amend Quantity Of An Existing Order
# 3 - Cancel An Existing Order
# 4 - Return Balance And Position

# request - (Action #, Trader ID, Additional Arguments)

# result - (Action #, Action Return)

# WE ASSUME 'AAPL' IS THE ONLY TRADED STOCK.


class Trader(MyThread):
    def __init__(self, id):
        super().__init__(id)
        self.book_position = 0
        self.balance_track = [1000000]
        self.order_type = 0
        # the traders each start with a balance of 1,000,000 and nothing on the books
        # each trader is a thread
    def join(self):
        if self.is_started == True:
            if self.balance_track[-1] <= 0:
                self.is_started = False

    def place_limit_order(self, quantity=None, price=None, side=None):
        # Make sure the limit order given has the parameters necessary to construct the order
        # It's your choice how to implement the orders that do not have enough information

        # The 'order' returned must be of type LimitOrder
        global order_id
        order = LimitOrder(order_id,self.id,quantity,price,side,time.time())
        order_id += 1
        self.order_type = order.type
        
        return order.type,self.id,order
        
        
        # Make sure you modify the book position after the trade
        # You must return a tuple of the following:
        # (the action type enum, the id of the trader, and the order to be executed)

    def place_market_order(self, quantity=None, side=None):
        # Make sure the market order given has the parameters necessary to construct the order
        # It's your choice how to implement the orders that do not have enough information
        
        # The 'order' returned must be of type MarketOrder
        
        # Make sure you modify the book position after the trade
        # You must return a tuple of the following:
        # (the action type enum, the id of the trader, and the order to be executed)
        global order_id
        order = MarketOrder(order_id,self.id,quantity,side,time.time())
        order_id += 1
        self.order_type = order.type
        
        return order.type,self.id,order       

    def place_ioc_order(self, quantity=None, price=None, side=None):
        # Make sure the ioc order given has the parameters necessary to construct the order
        # It's your choice how to implement the orders that do not have enough information
        
        # The 'order' returned must be of type IOCOrder
        
        # Make sure you modify the book position after the trade
        # You must return a tuple of the following:
        # (the action type enum, the id of the trader, and the order to be executed)
        global order_id
        order = IOCOrder(order_id,self.id,quantity,price,side,time.time())
        order_id += 1
        self.order_type = order.type
        
        return order.type,self.id,order

    def amend_quantity(self, quantity=None):
        # It's your choice how to implement the 'Amend' action where quantity is not given
        
        # You must return a tuple of the following:
        # (the action type enum, the id of the trader, and quantity to change the order by)
        return self.order_type, self.id, quantity

    def cancel_order(self):
        # You must return a tuple of the following:
        # (the action type enum, the id of the trader)
        self.book_position -= 1
        
        
        return self.order_type, self.id

    def balance_and_position(self):
        # You must return a tuple of the following:
        # (the action type enum, the id of the trader)
        return self.order_type,self.id

    def process_response(self, response):
        # Implement this function
        # You need to process each order according to the type (by enum) given by the 'response' variable
        if response.side == OrderSide.SELL:
            self.balance_track.append(self.balance_track[-1] + response.quantity * response.price)
            self.book_position -= response.quantity
        elif response.side == OrderSide.BUY:
            self.balance_track.append(self.balance_track[-1] - response.quantity * response.price)
            self.book_position += response.quantity
        # If the action taken by the trader is ambiguous you need to raise the following error
        else:
            raise UndefinedResponse("Undefined Response Received!")

    def random_action(self):
        # Implement this function
        # According to the status of whether you have a position on the book and the action chosen
        # the trader needs to be able to take a separate action
        
        # The action taken can be random or deterministic, your choice
        order_side_sign = random.randint(1,2)
        if order_side_sign == 1:
            order_side = OrderSide.BUY
        else:
            order_side = OrderSide.SELL
        order_type = random.randint(1,3)
        price = random.randint(1,5)
        quantity = random.randint(1,5)*100
        
        if order_type == 1:
            return self.place_limit_order(quantity=quantity,price=price,side=order_side)
        elif order_type == 2:
            return self.place_market_order(quantity=quantity,side=order_side)
        elif order_type == 3:
            return self.place_ioc_order(quantity=quantity,price=price,side=order_side)
        else:
            raise UndefinedResponse("Undefined Response Received!")            
        
        
    def run_infinite_loop(self):
        pass
        # The trader needs to continue to take actions until the book balance falls to 0
        # While the trader can take actions, it chooses from a random_action and uploads the action
        # to the exchange
        
        #The trader then takes any received responses from the exchange and processes it
        flag = 0
        while self.balance_track[-1] > 0 and flag == 0:
            order_type, self.id, order = self.random_action()
            for a,id,b in trader_to_exchange:
                #print(el)
                if self.id == id:
                    flag = 1
                    break
            if flag == 1:
                break
            else:
                trader_to_exchange.appendleft((order_type,self.id,order))
            if len(exchange_to_trader[self.id]) != 0:
                respond = exchange_to_trader[self.id].pop()
                self.process_response(respond)
            if self.balance_track[-1] == 0:
                self.is_started = False
            else:
                self.random_action()
            
        


class Exchange(MyThread):
    def __init__(self):
        super().__init__()
        self.balance = [1000000 for _ in range(100)]
        self.position = [0 for _ in range(100)]
        self.matching_engine = MatchingEngine()
        # The exchange keeps track of the traders' balances
        # The exchange uses the matching engine you built previously

    def place_new_order(self, order):
        # The exchange must use the matching engine to handle orders given
        results = []
        # The list of results is expected to contain a tuple of the follow form:
        # (Trader id that processed the order, (action type enum, order))
        
        # The exchange must update the balance of positions of each trader involved in the trade (if any)
        if order.type == OrderType.LIMIT:
            filled_order = self.matching_engine.handle_limit_order(order)
        elif order.type == OrderType.MARKET:
            filled_order = self.matching_engine.handle_market_order(order)
        elif order.type == OrderType.IOC:
            filled_order = self.matching_engine.handle_ioc_order(order)
        else:
            raise UndefinedResponse("Undefined Response Received!") 
        return results

    def amend_quantity(self, id, quantity):
        # The matching engine must be able to process the 'amend' action based on the given parameters
        
        # Keep in mind of any exceptions that may be thrown by the matching engine while handling orders
        # The return must be in the form (action type enum, logical based on if order processed)
        return (exchange_to_trader[id][0].type,self.matching_engine.amend_quantity(id, quantity))

    def cancel_order(self, id):
        # The matching engine must be able to process the 'cancel' action based on the given parameters
        
        # Keep in mind of any exceptions that may be thrown by the matching engine while handling orders
        # The return must be in the form (action type enum, logical based on if order processed)
        return (exchange_to_trader[id][0].type,matching_engine.cancel_order(id))

    def balance_and_position(self, id):
        # The matching engine must be able to process the 'balance' action based on the given parameters
        
        # The return must be in the form (action type enum, (trader balance, trader positions))
        return exchange_to_trader[id][0].type,(self.balance[id],self.position[id])

    def handle_request(self, request):
        global exchange_to_trader
        request_type, trader_id, order = request
        placed_order = self.place_new_order(order)
        # The exchange must be able to process different types of requests based on the action
        # type given using the functions implemented above
        
        # You must raise the following exception if the action given is ambiguous
        if len(placed_order) != 0:
            for fid, forder in placed_order:
                if forder.side == OrderSide.SELL:
                    self.balance[fid] += forder.price * forder.quantity
                    self.position[fid] -= forder.quantity
                    exchange_to_trader[fid].appendleft(forder)
                elif forder.side == OrderSide.BUY:
                    self.balance[fid] -= forder.price * forder.quantity
                    self.position[fid] += forder.quantity
                    exchange_to_trader[fid].appendleft(forder)
            # If the action taken by the trader is ambiguous you need to raise the following error
            else:
                raise UndefinedResponse("Undefined Response Received!")

    def run_infinite_loop(self):
        return
        # The exchange must continue handling orders as orders are issued by the traders
        # A way to do this is check if there are any orders waiting to be processed in the deque
        
        # If there are, handle the request using the functions built above and using the
        # corresponding trader's deque, return an acknowledgement based on the response
        while len(trader_to_exchange) != 0:
            order = trader_to_exchange.popleft()
            self.handle_request(order)


if __name__ == "__main__":

    trader = [Trader(i) for i in range(100)]
    exchange = Exchange()

    exchange.start()
    for t in trader:
        t.start()

    exchange.join()
    for t in trader:
        t.join()

    sum_exch = 0
    for t in MyThread.list_of_threads:
        if t.id == "NoID":
            for b in t.balance:
                sum_exch += b

    print("Total Money Amount for All Traders before Trading Session: " + str(sum_exch))

    for i in range(10000):
        thread_active = False
        for t in MyThread.list_of_threads:
            if t.is_started:
                t.run_infinite_loop()
                thread_active = True
        if not thread_active:
            break

    sum_exch = 0
    for t in MyThread.list_of_threads:
        if t.id == "NoID":
            for b in t.balance:
                sum_exch += b

    print("Total Money Amount for All Traders after Trading Session: ", str(int(sum_exch)))